# EDA

In [1]:
import pandas as pd

# 데이터 로드
url = 'https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv'
df = pd.read_csv(url)

# 기본 정보 확인
print(df.head())
print(df.info())
print(df.describe())

# 결측치 확인
print("결측치 수:\n", df.isnull().sum())

# Survived 클래스별 분포
print("생존자 수:\n", df['Survived'].value_counts())


   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN        S  
<c

# Preprocessing

In [2]:
# Age 결측치는 중앙값으로 채우기
df['Age'] = df['Age'].fillna(df['Age'].median())

# Embarked 결측치는 최빈값으로 채우기
df['Embarked'] = df['Embarked'].fillna(df['Embarked'].mode()[0])

# Sex, Embarked → 숫자 인코딩
df['Sex'] = df['Sex'].map({'male': 0, 'female': 1})
df['Embarked'] = df['Embarked'].map({'S': 0, 'C': 1, 'Q': 2})

# 필요 없는 컬럼 제거
df.drop(columns=['Name', 'Ticket', 'Cabin'], inplace=True)

# 피처와 타겟 설정

In [3]:
from sklearn.model_selection import train_test_split

# 피처와 타겟 설정
X = df[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']]
y = df['Survived']

# 학습/테스트 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 모델 훈련 및 평가

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, confusion_matrix

# 로지스틱 회귀
logreg = LogisticRegression(solver='liblinear')
logreg.fit(X_train, y_train)
logreg_preds = logreg.predict(X_test)
print("Logistic Regression 정확도:", round(accuracy_score(y_test, logreg_preds), 3))
print("Confusion Matrix (Logistic Regression):\n", confusion_matrix(y_test, logreg_preds))

# 나이브 베이즈
gnb = GaussianNB()
gnb.fit(X_train, y_train)
gnb_preds = gnb.predict(X_test)
print("Naive Bayes 정확도:", round(accuracy_score(y_test, gnb_preds), 3))
print("Confusion Matrix (Naive Bayes):\n", confusion_matrix(y_test, gnb_preds))


Logistic Regression 정확도: 0.799
Confusion Matrix (Logistic Regression):
 [[97 13]
 [23 46]]
Naive Bayes 정확도: 0.788
Confusion Matrix (Naive Bayes):
 [[93 17]
 [21 48]]


## 모델 성능 비교 및 요약

타이타닉 탑승객 데이터를 바탕으로 Logistic Regression와 Gaussian Naive Bayes 모델을 비교 분석하였다. 두 모델 모두 이진 분류 문제 해결에 널리 사용되는 알고리즘으로, 실제 데이터에 적용해 성능을 확인해 보았다.

### 정확도 및 혼동 행렬

| 모델                    | 정확도 | 오탐(False Positive) | 미탐(False Negative) |
|-------------------------|--------|----------------------|-----------------------|
| Logistic Regression     | 0.799  | 13                   | 23                    |
| Gaussian Naive Bayes    | 0.788  | 17                   | 21                    |

- Logistic Regression Confusion Matrix
```
  [[97 13]
   [23 46]]
```


- Naive Bayes Confusion Matrix
  ```
  [[93 17]
   [21 48]]
  ```

### 요약 및 해석

- **로지스틱 회귀**는 전체 정확도 면에서 약간 앞섰으며, 오탐/미탐의 균형도 상대적으로 안정적이었다.
- **가우시안 나이브 베이즈**는 속도나 단순성 면에서 장점이 있으나, 입력 특성 간의 독립성을 가정하기 때문에 실제 데이터에서는 다소 성능 저하가 발생할 수 있다.
- 전체적으로는 로지스틱 회귀가 더 신뢰할 수 있는 예측 결과를 보여주었다.

### 개선 아이디어

- 다른 분류 모델(SVM, 랜덤 포레스트 등)도 실험하여 비교 성능을 분석해 볼 수 있다.
- 특성 공학(feature engineering)을 통해 추가적인 정보를 반영하면 정확도를 높일 수 있다.
- 교차 검증과 하이퍼파라미터 튜닝을 적용해 일반화 성능을 향상시킬 여지도 있다.
